In [1]:
# Goal:visualize ensembles
# save last snapshot then save ensemble
%load_ext autoreload
%autoreload 2
# store files with visualization

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import matplotlib
matplotlib.use('Agg',warn=False, force=True)
from matplotlib import pyplot as plt
print("Switched to:",matplotlib.get_backend())
import matplotlib.pyplot as plt
import sys
#sys.path.insert(0, "/home/ekb2154/data/libraries/dgp_paninski/etc/dgp_tools/")
from deepgraphpose.models.fitdgp_util import get_snapshot_path
from deepgraphpose.models.eval import plot_dgp
from deeplabcut.utils import auxiliaryfunctions
from pathlib import Path
import os

Switched to: agg


/Users/taigaabe/anaconda3/envs/dgp/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


#Check which ensembles are the last per run

ModuleNotFoundError: No module named 'handle_iterations'

In [ ]:
# from moviepy.editor import VideoFileClip
# clip = VideoFileClip("/share/ctn/users/ekb2154/data/datasets/tracki/iblfingers/model_data/iblfingers-kelly-2030-01-01/videos/ibl60left.mp4")
# Math writeup? we already 90% of content but need to add for directpose

#Make videos comparing these frames

In [39]:
"""
Plot traces for ensembles and compare the outputs
Fix dgp_plot so that it actually does what it is supposed to do
Actually use ensembles for comparison
"""
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import os
os.environ['DLClight'] = 'True'
from deepgraphpose.models.eval import load_pose_from_dlc_to_dict
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
from moviepy.editor import VideoFileClip
from matplotlib import animation

task="ibl1"
#full_dir = '/datahd2a/datasets/tracki/{}'.format(task)
#full_dir = '/data/datasets/tracki' #/{}'.format(task)
#task="iblfingers"
#full_dir = "/share/home/ekb2154/data/datasets/tracki/{}/model_data/".format(task)
full_dir = "/Users/taigaabe/neurocaas_ensembles/data/"
#task="iblright"
scorer="kelly"
# new dates for ensemble
ranges = range(1,5)
dates= ["{}".format(ii) for ii in ranges]
run_names= ["run {}".format(ii) for ii in ranges]
frame_range = np.arange(0, 1000).astype('int')
n_frames = frame_range.size
# video_name = 'ibl60left'  #['ibl60left', 'ibl60right']
#video_name = 'left_movieflip'
video_name = "ibl1"
#moviefile = "/home/ekb2154/data/libraries/dgp_paninski/etc/ensembles/iblvideos/" + video_name + '.mp4'#project_dir / "videos" / ("{}.mp4".format(video_name))
moviefile = "/Users/taigaabe/neurocaas_ensembles/data/1/videos/{}.mp4".format(video_name)
snapshot =''
outname = '{}'.format(task)
figname = 'comparison_{}.mp4'.format(outname)
#%% make video comparing these 4 traces:
# make plot comparing them
xrs,yrs = [], []
for date_ in dates:
    project_dir = Path(full_dir)/"{}".format(date_)
    print("\n\n {}".format(project_dir))
    video_files = os.listdir(project_dir / "videos_pred" )
    video_files = [video_file_ for video_file_ in video_files if '.csv' in video_file_]
    video_files = [video_file_ for video_file_ in video_files if video_name in video_file_]
    #import pdb; pdb.set_trace()
    assert  len(video_files) == 1
    #print(video_files)
    label_file = str(project_dir / "videos_pred" / video_files[0])
    #label_file = project_dir / "videos_pred" / ("{}_labeled{}.csv".format(video_name, snapshot))
    # load labels
    labels = load_pose_from_dlc_to_dict(label_file)
    xr = labels['x']  # T x D
    yr = labels['y']  # T x D

    xr = xr[frame_range, :]
    yr = yr[frame_range, :]
    xrs.append(xr)
    yrs.append(yr)

    #moviefile = project_dir / "videos" / ("{}.mp4".format(video_name))



 /Users/taigaabe/neurocaas_ensembles/data/1


 /Users/taigaabe/neurocaas_ensembles/data/2


 /Users/taigaabe/neurocaas_ensembles/data/3


 /Users/taigaabe/neurocaas_ensembles/data/4


In [113]:
#%%
nj = xr.shape[1]
num_traces = len(xrs)
bodyparts = ['part {}'.format(ii) for ii in range(nj)] # replace w real name
#%%
print('Init video')
video_clip = VideoFileClip(str(moviefile))
fps = video_clip.fps
n_frames = min(n_frames, np.ceil(video_clip.fps * video_clip.duration).astype('int'))
frame_init = frame_range[0] / fps
frame_stop = frame_range[-1] / fps
# video_clip = video_clip.subclip(0, (n_frames / fps)) # it didnt take into account the range
video_clip = video_clip.subclip(frame_init, frame_stop) # it didnt take into account the range

Init video


In [58]:
ensembledict = {"run{}".format(i):np.stack((xrs[i],yrs[i]),axis = 1) for i in range(len(xrs))}
print(ensembledict.keys())


dict_keys(['run0', 'run1', 'run2', 'run3'])


In [110]:
t = 77 # 74
z = 4
colors = ["red","blue","green","purple"]

plt.imshow(video_clip.get_frame(t/video_clip.fps))
# for z in range(4):
#     for i in range(4):
#         plt.plot(*ensembledict["run{}".format(i)][t-z,:,:],"o",markersize = 1,linestyle = None,color = colors[i])

for i in range(4):
    for part in range(4):
        plt.plot(*ensembledict["run{}".format(i)][t-z:t+1,:,part].T,"o",markersize = 1,linestyle = None,color = colors[i],alpha = 0.3)
for i in range(4):
    plt.plot(*ensembledict["run{}".format(i)][t,:,:],"o",linestyle = 'None',markersize = 3,label = "run{}".format(i),color = colors[i])
plt.axis("off")
plt.legend()
plt.savefig("data/exampleframe{}.png".format(t))
plt.close()

In [133]:
from moviepy.video.io.bindings import mplfig_to_npimage
from moviepy.editor import VideoClip
colors = ["red","blue","green","purple"]
scalerate = 0.5
duration = video_clip.duration*scalerate
z = 4
fig,ax = plt.subplots()
def make_frame(tsecs):
    t = int(tsecs*video_clip.fps)
    ax.clear()
    ax.imshow(video_clip.get_frame(t/video_clip.fps))
    for i in range(4):
        for part in range(4):
            ax.plot(*ensembledict["run{}".format(i)][t-z:t+1,:,part].T,"o",markersize = 1,linestyle = None,color = colors[i],alpha = 0.3)
    for i in range(4):
        ax.plot(*ensembledict["run{}".format(i)][t,:,:],"o",linestyle = 'None',markersize = 3,label = "run{}".format(i),color = colors[i])
    plt.axis("off")  
    plt.legend()
    return mplfig_to_npimage(fig)

animation = VideoClip(make_frame,duration = duration)
animation.write_videofile(filename = "data/fullvideo.avi",codec = "png",fps = video_clip.fps*scalerate)


[MoviePy] >>>> Building video data/fullvideo.avi
[MoviePy] Writing video data/fullvideo.avi







  0%|          | 0/250 [00:00<?, ?it/s]




  0%|          | 1/250 [00:00<00:25,  9.95it/s]




  1%|          | 2/250 [00:00<00:27,  9.14it/s]




  1%|          | 3/250 [00:00<00:28,  8.63it/s]




  2%|▏         | 4/250 [00:00<00:30,  8.09it/s]




  2%|▏         | 5/250 [00:00<00:35,  6.92it/s]




  2%|▏         | 6/250 [00:00<00:43,  5.55it/s]




  3%|▎         | 7/250 [00:01<00:39,  6.09it/s]




  3%|▎         | 8/250 [00:01<00:35,  6.84it/s]




  4%|▎         | 9/250 [00:01<00:32,  7.48it/s]




  4%|▍         | 10/250 [00:01<00:30,  7.91it/s]




  4%|▍         | 11/250 [00:01<00:28,  8.30it/s]




  5%|▌         | 13/250 [00:01<00:26,  8.91it/s]




  6%|▌         | 14/250 [00:01<00:27,  8.70it/s]




  6%|▌         | 15/250 [00:01<00:27,  8.64it/s]




  7%|▋         | 17/250 [00:02<00:25,  9.27it/s]




  7%|▋         | 18/250 [00:02<00:24,  9.37it/s]




  8%|▊         | 20/250 [00:02<00:23,  9.77it/s]




  9%|▉         | 22/250 [00:02<00:22,  9.93it/s]




 10%|▉

 98%|█████████▊| 246/250 [00:40<00:00,  6.04it/s]




 99%|█████████▉| 247/250 [00:40<00:00,  6.59it/s]




 99%|█████████▉| 248/250 [00:41<00:00,  6.81it/s]




100%|█████████▉| 249/250 [00:41<00:00,  4.94it/s]




100%|██████████| 250/250 [00:41<00:00,  5.75it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: data/fullvideo.avi 



In [ ]:
#%%
from itertools import chain
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams.update({  # 'font.sans-serif' : 'Helveltica',
    'axes.labelsize': 9,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'axes.titlesize': 11})

#%%
# coment out y 
num_coordinates = 1 # x and y or just 2
fig2 = plt.figure(constrained_layout=False, figsize=(11, 10))
widths = [2, 7]
heights = np.ones(num_coordinates * nj)
num_cols = 2
num_rows = num_coordinates*nj
spec2 = fig2.add_gridspec(ncols=num_cols, nrows=num_rows, width_ratios=widths,
                          height_ratios=heights)

framerows = num_rows// num_traces
#%% Plot
markersize = 2
markersize2 = 5
from matplotlib.lines import Line2D
color_class = plt.cm.ScalarMappable(cmap="cool")
colors = color_class.to_rgba(np.linspace(0, 1, nj))
markers = Line2D.filled_markers[:num_traces]

#%%
frame_0name = 0
frame = video_clip.get_frame(0)
tidx = 0
colors_traces= ['C{}'.format(ii) for ii in range(num_traces)]

imagesfig = [[]]*num_traces
imagesmakers = [[]]*num_traces*nj
xtracesmarkers = [[]]*num_traces*nj
ytracesmarkers = [[]]*num_traces*nj
frametitles = [[]]*num_traces

count= 0
count1 = 0
for ss_idx in range(num_traces):
    # for the image
    f2_ax1 = fig2.add_subplot(spec2[ss_idx*framerows:ss_idx*framerows+ framerows, 0])
    if ss_idx == 0:
        frametitles[ss_idx] = f2_ax1.set_title('Frame {} \n {} '.format(frame_0name, run_names[ss_idx]),
                                               color=colors_traces[ss_idx], weight='bold')
    else:
        frametitles[ss_idx] = f2_ax1.set_title('{}'.format(run_names[ss_idx]),
                                               color=colors_traces[ss_idx], weight='bold')
    imagesfig[ss_idx] = f2_ax1.imshow(frame)
    dgp_frames = []
    for njj in range(nj):
        dxs_dgp = xrs[ss_idx][tidx]
        dys_dgp = yrs[ss_idx][tidx]
        dgp_frame, = f2_ax1.plot(dxs_dgp[njj], dys_dgp[njj], c=colors[njj],
            marker=markers[ss_idx], ms=markersize, )
        imagesmakers[count] = dgp_frame
        count+=1
    # x traces
    for njj in range(nj):
        f2_ax2 = fig2.add_subplot(spec2[num_coordinates * njj, 1:])
        f2_ax2.set_yticks([])
        f2_ax2.set_xticks([])
        # plot trace
        f2_ax2.plot(xrs[ss_idx][:, njj], c=colors_traces[ss_idx], linewidth=1.5, linestyle='--')
        dgp_mtx, = f2_ax2.plot(tidx, dxs_dgp[njj], c=colors_traces[ss_idx], marker=markers[ss_idx], markersize=markersize2,
                               markerfacecolor=None)
        #f2_ax2.set_ylabel('x')
        f2_ax2.set_xlim([0, n_frames])
        f2_ax2.set_ylabel('{}'.format(bodyparts[njj]), color=colors[njj], weight='bold')

        xtracesmarkers[count1] = dgp_mtx
        count1+=1


    # y traces
    if num_coordinates ==2:
        for njj in range(nj):
            f2_ax2 = fig2.add_subplot(spec2[num_coordinatess * njj + 1, 1:])
            f2_ax2.set_yticks([])
            f2_ax2.set_xticks([])
            # plot trace
            #f2_ax2.plot(yrs[ss_idx][:, njj], c=colors[njj], linewidth=1.5, linestyle='--')
            f2_ax2.plot(yrs[ss_idx][:, njj], c=colors_traces[ss_idx], linewidth=1.5, linestyle='--')
            dgp_mty, = f2_ax2.plot(tidx, dys_dgp[njj], c=colors_traces[ss_idx], marker=markers[ss_idx], markersize=markersize2,
                                   markerfacecolor=None)
            f2_ax2.set_ylabel('y')
            f2_ax2.set_xlim([0, n_frames])

            ytracesmarkers[ss_idx].append(dgp_mty)
    

allmyparts = imagesfig + imagesmakers + xtracesmarkers
    
plt.tight_layout()
plt.show()

In [ ]:
#%%
def init():
    return allmyparts

def animate3(tidx):
    frame = video_clip.get_frame(tidx * 1.0 / fps)
    for ss_idx in range(num_traces):
        if (ss_idx == 0) or (ss_idx == num_traces-1):
            pass
            #print('ss', ss_idx)
        dxs_dlc = xrs[ss_idx][tidx,:]
        dys_dlc = yrs[ss_idx][tidx,:]
        if ss_i|dx == 0:
            frametitles[ss_idx].set_text("Frame {} \n {} ".format(tidx, run_names[ss_idx]))
        imagesfig[ss_idx].set_array(frame)
    count=0
    count1 = 0
    for ss_idx in range(num_traces):
        for nji in range(nj):
            dxs_dlc = xrs[ss_idx][tidx, nji]
            dys_dlc = yrs[ss_idx][tidx, nji]
            imagesmakers[count].set_data(dxs_dlc, dys_dlc)
            xtracesmarkers[count].set_data(tidx, dxs_dlc)
            count +=1
            #count1 +=1
    return allmyparts

print('NFRAMES +{}'.format(n_frames))
anim = animation.FuncAnimation(fig2, animate3,  init_func=init,
                               frames=10, interval=20, blit=True,
                              repeat=False)

In [ ]:
from IPython.display import HTML
HTML(anim.to_jshtml())

In [ ]:
# now based on this calculate best path
import time
start = time.time()
anim.save(figname)
print(time.time() - start)
plt.close(fig2)
#%%
video_clip.close()